<a href="https://colab.research.google.com/github/stevenpollack/camelcup/blob/master/camelUp_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SKIP DOWN TO ANNOTATED TEXT FOR SIMULATION CODE

In [0]:
# numpy for random shuffling
# collections for deque DS
import numpy as np
import pandas as pd
import collections, warnings, time

In [0]:
class Dice:
  faces = [1,2,3]
  def __init__(self, color='', face=-1):
    self.color = color
    self.face = face

  def __str__(self):
    if self.color: # '' returns False
      return "{} {}".format(self.color, self.face)
    else:
      return "{}".format(self.face)
  
  def roll(self, verbose=False):
    self.face = np.random.choice(self.faces, size=1)[0]
    if verbose:
      print("Rolled {}".format(self.face))
    return self.face

class Camel:
  def __init__(self, color, location=-1):
    self.color = color
    self.dice = Dice(color)
    self.location = location
    self.already_moved = False

  def to_dict(self):
    return {
        "color": self.color,
        "location": self.location,
        "already_moved": self.already_moved
    }
    
  def __repr__(self):
    return "{} camel".format(self.color)
  def __str__(self):
    return "{} camel is on space #{}".format(self.color, self.location)

In [0]:
class Space:
  def __init__(self, effect=0, location=-1):
    # put all instance specific attributes in initializer!
    self.camels = collections.deque()
    self.effect = effect
    self.location = location
    
  def __str__(self):
    base_string = "Space has location {} and ".format(self.location)
    if self.camels:
      return base_string + "{}".format(self.camels)
    elif self.effect == 1 or self.effect == -1:
      return base_string + "a modifier of {}".format(self.effect)
    else:
      return "Space is empty"
  
  def to_dict(self):
    output = {"location": self.location} 
    if self.camels:
      output["camels"] = [camel.to_dict() for camel in self.camels]
    elif self.effect != 0:
      output["modifications"] = {"location": self.location, "effect": self.effect}
    return output

    camels = []
    modifications = {}
    return {'location': self.location}
  def __repr__(self):
    return "<Space: location {}, {}, and effect {}>".format(self.location, self.camels, self.effect)

  def add_camels_on_top(self, camels):
    # reverse the camel stack to preserve order when left-appending
    camels.reverse()
    for camel in camels:
      self.camels.appendleft(camel)
      camel.location = self.location # set the camel's location to this new Space
    
  def add_camels_to_bottom(self,camels):
    assert (self.effect == 0), "You cannot add a Camel to location {} since it has modification {}".format(self.location, self.effect)

    for camel in camels:
      self.camels.append(camel)
      camel.location = self.location # set the camel's location to this new Space

In [0]:
class Board:
  available_colors = ['Orange', 'Green', 'Blue', 'White', 'Yellow']  
  def __init__(self, num_of_spaces = 16):
  # create empty board
    self.moved_camels, self.ready_camels = collections.deque(), collections.deque()
    self.leg_ranking = collections.deque() # this is an ordered list of camels
    self.final_ranking = [] # this will be an ordered list of camels, upon game end
    self.num_of_spaces = num_of_spaces
    self.game_over = False
    self.sim_results = []
    self.spaces = []
    for i in range(self.num_of_spaces+1): # we want the 0th space to be reserved for the winners
      new_space = Space(location=i)
      self.spaces.append(new_space)

  def initialize_camels(self, camel_colors):
    # back in some color checking here, later
    # for now assume camel_colors is an array of strings
    for color in camel_colors:
      camel = Camel(color)
      self.camels.append(camel)
      roll = camel.dice.roll() # there's an 1 vs 0 issue on dice rolls versus indexing
      camel.location = roll
      space = self.spaces[roll]
      space.add_camel_on_top(camel)
  
  def load_from_dict(self, dict):
    """ 
      general rule will be that the dictionaries are written in such as way
      that if two camels occupy the same space, their order will be 
      left->right == top->bottom. E.g. [red, blue] => red sits atop blue.
    """
    # build error protection later
    # infer number of players from dict:
    num_players = 0

    for camel_dict in dict['camels']:
      num_players += 1
      camel = Camel(camel_dict['color'], camel_dict['location'])
      # add camel to the space, building the stack from top-down
      self.spaces[camel.location].camels.append(camel)
      # sort camels depending on whether they've moved or not
      if camel_dict['already_moved']:
        self.moved_camels.append(camel)
      else:
        self.ready_camels.append(camel)
    
    for mod_dict in dict['modifications']:
      assert (mod_dict['effect'] in [-1, 0 , 1]), "Tile effects may only take vaues +/-1 or 0. Modification was set to {}".format(mod_dict['effect'])
      self.spaces[mod_dict['location']].effect = mod_dict['effect']
    
    return self
  
  def calculate_camel_ranking(self, parseable_ranking=True):
    # if parseable_ranking is true, we store the rankings as the camel colors,
    # otherwise we store the entire camel object

    # for a leg:
    self.leg_ranking = []
    camel_stacks = collections.deque()

    # iterate along the board and aggregate the camel_stacks
    [camel_stacks.appendleft(space.camels) for space in self.spaces[1:] if space.camels]

    # check podium
    if self.spaces[0].camels:
      camel_stacks.appendleft(self.spaces[0].camels)


    # flatten ranking
    for camel_stack in camel_stacks:
      for camel in camel_stack:
        if parseable_ranking:
          self.leg_ranking.append(camel.color)
        else:
          self.leg_ranking.append(camel) 

    return self.leg_ranking;
  
  def simulate_one_round(self, initial_state, sims=1000, seed=-1, append_results=True):
    
    if append_results:
      sim_results = self.sim_results
    else:
      sim_results = []

    if seed != -1:
      np.random.seed(seed)

    for i in range(sims):
      # start fresh
      self.__init__()
      self.load_from_dict(initial_state)

      # sim and store results
      sim_result = self.playout_one_round()
      #print(sim_result)
      sim_results.append(sim_result)
       
    self.sim_results = sim_results # save results in Board object for future appending
    return sim_results
  
  def simulate_rest_of_game(self, sims=1000):
    return;

  def to_dict(self):
    """ output should match that of the dictionaries we feed to load_from_dict
    """
    output = {"camels": []}
    mods = []
    for space in self.spaces:
      space_dict = space.to_dict()
      
      if space.camels:
        [output['camels'].append(camel) for camel in space_dict['camels']] # camels is necessarily an array
      elif space.effect != 0:
        mods.append(space_dict['modifications'])
   
    # append modifications, if any exist
    if mods:
      output['modifications'] = mods

    return output

  def roll_and_resolve_camel(self, camel, verbose=False):
    if camel.already_moved:
      warnings.warn("{} has already moved!".format(camel))
      return;

    roll = camel.dice.roll(verbose=verbose)
    camel.already_moved = True
    future_location = camel.location + roll

    # we need to figure out if this camel has camels on its back
    current_space = self.spaces[camel.location]
    camel_index = current_space.camels.index(camel)
    # simultaneous make camel stack and remove camels from current Space
    camel_stack = [current_space.camels.popleft() for x in range(camel_index+1)] 
    if verbose:
      print(camel_stack)

    if future_location > self.num_of_spaces:  # game is over

      final_space = self.spaces[0] #this is the "podium"
      final_space.add_camels_on_top(camel_stack)
      
      # need to end the round
      self.calculate_camel_ranking()
      self.game_over = True

      if verbose:
        print("game ending!")

    else: # no camels have crossed the finish line

        # simulate bouncing forwards or backwards across potentially many mirages or oases
      final_space = self.spaces[future_location]
      tile_effect = 0 # this will get updated in the loop, if necessary. It's == final_space.effect

      while final_space.effect != 0:
        # check for tile effects
        tile_effect = final_space.effect
        assert (tile_effect in [-1, 0, 1]), "the effect property of a Space must be either +/-1 or 0"
        future_location += tile_effect # update
        final_space = self.spaces[future_location]
    
      if tile_effect >= 0: # pop camel atop stack
        final_space.add_camels_on_top(camel_stack)
      
      elif tile_effect == -1: # append camel to bottom of stack
        final_space.add_camels_to_bottom(camel_stack)

    return;


  def playout_one_round(self, seed=-1, verbose=False, output_rankings=True, parseable_ranking=True):
    # if there are no camels that can move, this is all moot
    if not self.ready_camels:
      return self.to_dict();

    # shuffle the camels that have yet to move
    if seed != -1:
      np.random.seed(seed)
    np.random.shuffle(self.ready_camels)

    # roll each dice, one-by-one, resolving effects:
    for camel in self.ready_camels:
      if verbose:
        print("Rolling for {}".format(camel))
      self.roll_and_resolve_camel(camel, verbose=verbose)
      self.moved_camels.append(camel)
      if self.game_over:
        break

    # reset read_camels stack to empty deque and flip state of board
    if not self.game_over:
      self.calculate_camel_ranking(parseable_ranking)

    self.ready_camels = collections.deque()

    if output_rankings:
      return self.leg_ranking
    else:
      return self.to_dict()


  def __str__(self):
    print("*** Camel Up! ****\n---------------------------------------------")
    for space in self.spaces:
      print(space)
    print("----------------------------------------------")
    print("The following camels have moved: {}".format(self.moved_camels) + 
          "\nThe remaining camels can still move: {}\n".format(self.ready_camels))
    return "";


if False:
  test_board = Board(num_players=3)
  test_board.load_from_dict(test_dict)
  print(test_board.calculate_camel_ranking())
  #print(test_board)
  test_board.playout_one_round(seed=1337,verbose=True)

  print(test_board.calculate_camel_ranking())
  test_board.to_dict()

# MODIFY THE CODE BELOW TO RUN SIMULATION:
`test_dict` is a dictionary (JSON) object with keys `camels` and `modifications`. Both of which are arrays containing more dictionaries.

The template for the objects inside these arrays is provided below. You'll want to **run all cells** before executing the block below (CTRL+F9 or `Runtime > Run All`).

`location` can be any integer from [1,16]:
 * 1 being the first tile on the board,
 * 16 being the last tile before the finish line
 * 0 is reserved for the 'podium', don't use it!

`effect` is either -1, 0, 1, indicating whether a space has a desert tile (-1),
oasis tile (+1), or can accept camels (0). 

In [11]:

test_dict = {"camels": [{"color": "orange", "location": 5, "already_moved": True},
                         {"color": "blue", "location": 2, "already_moved": False},
                         {"color": "yellow", "location": 2, "already_moved": False}],
                         #{"color": "green", "location":1, "already_moved": False},
                         #{"color": "white", "location":1, "already_moved": False}],
               "modifications":[{"location": 15, "effect": -1}, {"location": 16, "effect": +1}]}
initial_state = test_dict
sims = 100000
test_sim = Board()
t0 = time.time()
sim_results = test_sim.simulate_one_round(initial_state, sims=sims)
#sim_results = test_sim.simulate_one_round(initial_state, sims=sims) 
t1 = time.time()
print(t1-t0)
# place simulation results into a data-frame and return probabilities,
# sorted so that the most likely camel to get 1st place is on top
result_df = pd.DataFrame(sim_results)
dt = result_df.apply(lambda col: col.astype("category"), axis=0)
dt = dt.apply(lambda col: col.value_counts(normalize=True), axis=0)
dt.rename(columns={0: 'first_place', 1: 'second_place'}, inplace=True)
dt.sort_values(dt.columns[0],ascending=False)

9.519516468048096


,first_place,second_place,2
blue,0.55505,0.16741,0.27754
orange,0.27798,0.50040,0.22162
yellow,0.16697,0.33219,0.50084


In [6]:
(2/9)*(14/18) + (1/9)*(5/18 + 4/3)

0.35185185185185186

In a 2 camel race between _T_ and _B_ where _T_ sits atop _B_: $$P(T \text{ wins}) = \frac{2}{3}$$.


# IGNORE BELOW
We've got an interconnectivity between a few objects:

` Camel ` owns a dice and sits on a `Space` on the `Board`

`Camel` has a few attributes/methods:
* `color`: either red, blue, green, white, etc.
* `space`: the index in `Board` of the space where the camel is located
* `dice`: the `Dice` object that displays [1..6] or `Null`, depending on if the dice has been rolled this round, or not.

`Board` represents the playing field and has the following fields:
* `spaces`: an array of `Space` objects.
* `dices`: an array of `Dice` objects.

A `Space` represents the area on the `Board` that can be occupied by an oasis/desert (+1/-1)  XOR a stack of `Camel` objects. It has attributes:
* `camels`: a deque of `Camel` objects (or `[]`).
* `effect`: an indicator if this space has been modified (yield either 0,1,-1).

`roll_dice()`
